**Problem set 1**
**Subject: IO**
**Start Date 01/12/2025**

**Problem 1**

In [2]:
using CSV, DataFrames

#Input path
data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year2/term1/io/part2/data_ps1.csv" ##-->CHANGE JUST THIS PART MANUALLY!

#Output path
export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/" ##Change this manually too

main = CSV.read(data_path, DataFrame)

Row,yr,prefid,firmid,m,k,age,acquired,cost_m,q,rev,l,cost_l,cost_k
,Int64,Int64,Int64,Int64?,Int64?,Int64,Int64,Float64?,Float64,Float64,Int64?,Float64?,Float64?
1,1917,1,1,174866,120960,0,0,4.81021e5,1.55838e5,385355.0,1045968,49963.2,missing
2,1918,1,1,225000,120600,1,0,9.72158e5,412452.0,2.01e6,1359765,84420.0,missing
3,1919,1,1,225266,199248,2,0,1.16798e6,8.82063e5,5.14022e6,4956336,4.60354e5,missing
4,1920,1,1,383289,306440,3,0,2.14914e6,1.11663e6,5.15899e6,5278592,215085.0,missing
5,1896,1,2,33580,5772,18,0,38996.5,24626.3,55610.1,522144,2925.96,missing
6,1896,4,4,502159,112346,4,0,5.83157e5,3.97768e5,789222.0,8201556,53398.6,16698.8
7,1897,4,4,532756,144058,5,0,6.45381e5,461018.0,8.54804e5,8520094,78892.8,43161.2
8,1898,4,4,190027,49268,6,0,2.19785e5,150335.0,0.0,2707342,23877.5,30933.8
9,1896,16,5,843772,243090,7,0,9.79872e5,1.00331e6,1.35233e6,13692294,1.02388e5,39081.8


In [41]:
#1.a
using Statistics

#define variables

l= main.l
w= main.cost_l
rev= main.rev
k= main.k
r= main.cost_k
q= main.q


beta_l= ifelse.(rev.>0, (w) ./ (rev), missing) 

beta_l_total = mean(skipmissing(beta_l))
beta_l_std= std(skipmissing(beta_l))
beta_l_std= median(skipmissing(beta_l))



0.050841608736857224

In [46]:
#1.b 
using Statistics

total_cost = w .+  r

idx = findall(x -> !ismissing(x) && x > 0, total_cost)

beta_l_2 = (w[idx]) ./ total_cost[idx]


beta_l_total_2 = mean(beta_l_2)
beta_l_std_2= std(beta_l_2)
beta_l_std_2= median(beta_l_2)



0.5693124616887979

In [47]:
#1.c 

markup= rev[idx] ./ total_cost[idx]

markup_mean = mean(markup)
markup_std= std(markup)
markup_median= median(markup)


12.086569058219947